In [1]:
import sympy as sym

import numpy as np

import evol_dynamics

In [2]:
def invariant_distribution(M):

    eigenvalues, eigenvectors = np.linalg.eig(M.T)
    eigenvectors_one = eigenvectors[:, np.argmax(eigenvalues)]

    stationary = eigenvectors_one / eigenvectors_one.sum()

    return stationary.real

In [3]:
def invariant_distribution_analytically(M):
    size = M.shape[1]
    pi = sym.symbols(f"b_1:{size + 1}")
    ss = sym.solve(
        [sum(pi) - 1]
        + [a - b for a, b in zip(M.transpose() * sym.Matrix(pi), pi)],
        pi,
    )

    v_vector = sym.Matrix(
        [
            [ss[p] for p in pi],
        ]
    )

    return v_vector

### Invasion analysis of ALLD into GTFT

In [4]:
q = sym.symbols("q")

In [5]:
alld = (0, 0, 0)

gtft = (1, 1, q)

In [6]:
M = evol_dynamics.markov_chain_for_reactive_strategies(alld, gtft)

invariant_distribution_analytically(M)

Matrix([[0, 0, q, 1 - q]])

In [7]:
M = evol_dynamics.markov_chain_for_reactive_strategies(alld, alld)

invariant_distribution_analytically(M)

Matrix([[0, 0, 0, 1]])

In [8]:
M = evol_dynamics.markov_chain_for_reactive_strategies(gtft, alld)

invariant_distribution_analytically(M)

Matrix([[0, q, 0, 1 - q]])

In [9]:
M = evol_dynamics.markov_chain_for_reactive_strategies(gtft, gtft)

invariant_distribution_analytically(M)

Matrix([[1, 0, 0, 0]])

## Last two rounds

In [17]:
delta = sym.symbols("delta")

In [28]:
def ss_last_two_rounds(s1, s2, delta):
    M = evol_dynamics.markov_chain_for_reactive_strategies(s1, s2)
    
    v0 = np.array([s1[0] * s2[0], s1[0] * (1 - s2[0]), (1 - s1[0]) * s2[0], (1 - s1[0]) * (1 - s2[0])])
    
    A = sym.eye(4) - delta * M
    
    rhs = np.dot(v0, A.inv())
    
    v = sym.zeros(4, 4)
    
    for i in range(4):
        for j in range(4):
            v[i, j] = (1 - delta) * M[i, j] * (delta ** 2) * rhs[i]
            
    return v.reshape(1, 16)

In [32]:
v = ss_last_two_rounds(alld, gtft, delta)

In [33]:
vMM = ss_last_two_rounds(alld, alld, delta)
vMR = ss_last_two_rounds(alld, gtft, delta)
vRM = ss_last_two_rounds(gtft, alld, delta)
vRR = ss_last_two_rounds(gtft, gtft, delta)

In [491]:
vRM = ss_last_two_rounds(gtft, alld, delta)

In [613]:
vRM

Matrix([[0, 0, 0, 0, 0, -delta**2*q*(1 - delta)*(delta*q - delta + 1)/(delta - 1), 0, -delta**2*(1 - delta)*(1 - q)*(delta*q - delta + 1)/(delta - 1), 0, 0, 0, 0, 0, delta**2*q*(1 - delta)*(delta*q - delta)/(delta - 1), 0, delta**2*(1 - delta)*(1 - q)*(delta*q - delta)/(delta - 1)]])

In [493]:
vMR

Matrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -delta**2*q*(1 - delta)*(delta*q - delta + 1)/(delta - 1), -delta**2*(1 - delta)*(1 - q)*(delta*q - delta + 1)/(delta - 1), 0, 0, delta**2*q*(1 - delta)*(delta*q - delta)/(delta - 1), delta**2*(1 - delta)*(1 - q)*(delta*q - delta)/(delta - 1)]])

In [496]:
x[5, 10]

-delta**2*q*(1 - delta)*(delta*q - delta + 1)/((N - 1)*(delta - 1))

In [34]:
b, c = sym.symbols("b, c")

In [38]:
(1==1 & 2==1) | (1==1 & 1==1)

True

In [55]:
1 / (N - 1) * vRM[i1 - 1]

-delta**2*q*(1 - delta)*(delta*q - delta + 1)/((N - 1)*(delta - 1))

In [56]:
((i1==1 & i2==1) | (i1==2 & i2==3)  |(i1==3 &i2==2)   |(i1==4 & i2==4) |(i1==5 & i2==9) |(i1==6 & i2==11)| (i1==7 & i2==10) |(i1==8 & i2==12) |(i1==9 & i2==5)  |(i1==10 & i2==7)|(i1==11 & i2==6)|(i1==12 & i2==8)| (i1==13 & i2==13)|(i1==14 & i2==15)|(i1==15 & i2==14)|(i1==16 & i2==16))

False

In [219]:
x = sym.zeros(16,16)

k = 1

N = sym.symbols("N")

for i1 in range(1, 17):
    for i2 in range(1, 17):
        
        feasible = ((i1==1 and  i2==1)    |(i1==2 and  i2==3)
        |(i1==3 and i2==2)    |(i1==4 and  i2==4)
        |(i1==5 and  i2==9)   |(i1==6 and  i2==11)
        | (i1==7 and  i2==10) |(i1==8 and  i2==12) 
        |(i1==9 and  i2==5)   |(i1==10 and  i2==7)
        |(i1==11 and  i2==6)  |(i1==12 and  i2==8)
        | (i1==13 and  i2==13)|(i1==14 and  i2==15)
         |(i1==15 and  i2==14)|(i1==16 and  i2==16))

        first = 1 / (N - 1) * vRM[i1 - 1]

        second = (1 - 1 / (N - 1)) / (N - 2) / (N - 3) * ((k - 1) * (k - 2) * vRM[i1 - 1] * vMM[i2 - 1] +
                                                          (k - 1) * (N - k - 1) * vRM[i1 - 1] * vMR[i2 - 1] + (N - k - 1) * (k - 1) * vRR[i1 - 1] * vMM[i2 - 1]
                                                          + (N - k - 1) * (N - k - 2) * vRR[i1 - 1] * vMR[i2 - 1])
        
        if feasible:
            x[i1 - 1, i2 - 1] = first + second
        else:
            x[i1 - 1, i2 - 1] = second
    

In [220]:
beta = sym.symbols("beta")

In [221]:
def Rho(u, beta):

    Us = sym.zeros(16, 16)

    for i in range(16):
        for j in range(16):
#             print(int((j - 1) / 4), (j) % 4, int((i - 1) / 4), (i) % 4)
            Us[i, j] = (u[int((j) / 4)] + u[(j) % 4]) - (u[int((i) / 4)] + u[(i) % 4])

    Us = Us / 2
    
    rho = sym.zeros(16, 16);

    for i in range(16):
        for j in range(16):
            rho[i, j] = 1 / (1 + sym.exp(-beta * Us[i, j]))
            

            return rho

In [518]:
R, S, T, P = sym.symbols("R, S, T, P")

u = [R, S, T, P];

In [520]:
rho = Rho(u, beta)

In [521]:
1 / (np.exp(-1) + 1)

0.7310585786300049

In [522]:
sym.exp(-beta  * c) + 1

1 + exp(-beta*c)

In [615]:
f + s

(1 - 1/(N - 1))*(-delta**4*q*(delta*q - delta)/(1 + exp(-beta*(P/2 - R + T/2))) + delta**4*q*(delta*q - delta + 1)/(1 + exp(-beta*(-R + T))) + delta**4*(1 - q)*(delta*q - delta + 1)/(1 + exp(-beta*(P/2 - R + T/2))) - delta**4*(1 - q)*(delta*q - delta)/(1 + exp(-beta*(P - R)))) + (-delta**2*q*(delta*q - delta)/(1 + exp(-beta*(-S/2 + T/2))) + delta**2*q*(delta*q - delta + 1)/(1 + exp(-beta*(-S + T))) - delta**2*(1 - q)*(delta*q - delta)/2 + delta**2*(1 - q)*(delta*q - delta + 1)/(1 + exp(-beta*(-S/2 + T/2))))/(N - 1)

In [616]:
f2

(1 - 1/(N - 1))*(-delta**4*q*(delta*q - delta)/(1 + exp(-beta*(-P/2 + R - T/2))) + delta**4*q*(delta*q - delta + 1)/(1 + exp(-beta*(R - T))) + delta**4*(1 - q)*(delta*q - delta + 1)/(1 + exp(-beta*(-P/2 + R - T/2))) - delta**4*(1 - q)*(delta*q - delta)/(1 + exp(-beta*(-P + R))))

In [524]:
s2

(-delta**2*q*(delta*q - delta)/(1 + exp(-beta*(-b/2 - c/2))) + delta**2*q*(delta*q - delta + 1)/(1 + exp(-beta*(-b - c))) - delta**2*(1 - q)*(delta*q - delta)/2 + delta**2*(1 - q)*(delta*q - delta + 1)/(1 + exp(-beta*(-b/2 - c/2))))/(N - 1)

In [525]:
vMR.subs({q:1/5, delta:0.9})

Matrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.04536, 0.18144, 0, 0, 0.11664, 0.46656]])

In [526]:
vRM.subs({q:1/5, delta:0.9})

Matrix([[0, 0, 0, 0, 0, 0.04536, 0, 0.18144, 0, 0, 0, 0, 0, 0.11664, 0, 0.46656]])

In [527]:
vRR.subs({q:1/5, delta:0.9})

Matrix([[0.81, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [528]:
evol_dynamics.stationary_for_16_states((0, 0, 0), (1, 1, 1/5), .9).round(3)

array([-0.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.   , -0.   ,
        0.   ,  0.   ,  0.045,  0.181,  0.   ,  0.   ,  0.117,  0.467])

In [529]:
evol_dynamics.stationary_for_16_states((1, 1, 1/5), (0, 0, 0), .9).round(3)

array([-0.   , -0.   , -0.   , -0.   ,  0.   ,  0.045,  0.   ,  0.181,
       -0.   , -0.   , -0.   , -0.   ,  0.   ,  0.117,  0.   ,  0.467])

In [530]:
evol_dynamics.stationary_for_16_states((1, 1, 1/5), (1, 1, 1/5), .9).round(3)

array([0.81, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  ])

In [556]:
num1 = - (delta ** 4 * q * (delta * q - delta))

num2 = (delta ** 4 * q * (delta * q  - delta + 1))

num3 = delta ** 4 * (1 - q) * (delta * q  - delta + 1)

num4 = - delta ** 4 * (1 - q) * (delta * q - delta)



num21 = delta ** 2 * q * (delta * q - delta + 1) 

num22 = delta ** 2 * q * (delta * q - delta)

num23 = delta ** 2 * (1 - q) * (delta * q - delta)

num24 = delta ** 2 * (1 - q) * (delta * q - delta + 1)


f = (1 - 1/ (N - 1)) * (num1 / (sym.exp(-beta  * (P / 2 - R + T / 2)) + 1) + num2 / (sym.exp(-beta  * (-R + T)) + 1)
                       + num3 / (sym.exp(-beta  * (P / 2 - R + T / 2)) + 1) + num4 / (sym.exp(-beta  * (P - R)) + 1))


s = (1 / (N - 1)) * (- num22 / (sym.exp(-beta  * (- S / 2 + T / 2)) + 1) + num21 / (sym.exp(-beta  * (- S + T)) + 1)
                    - num23 / 2 + num24 / (sym.exp(-beta  * (- S / 2 + T / 2)) + 1))

In [555]:
s

(-delta**2*q*(delta*q - delta + 1)/(1 + exp(-beta*(-S/2 + T/2))) + delta**2*q*(delta*q - delta)/(1 + exp(-beta*(-S + T))) - delta**2*(1 - q)*(delta*q - delta)/2 + delta**2*(1 - q)*(delta*q - delta + 1)/(1 + exp(-beta*(-S/2 + T/2))))/(N - 1)

In [548]:
expr = (sum(sum(np.multiply(x, rho))))

In [549]:
expr.collect(delta - 1).collect((1 / N - 1))

-delta**4*q*(1 - 1/(N - 1))*(delta*q - delta)/(1 + exp(-beta*(P/2 - R + T/2))) + delta**4*q*(1 - 1/(N - 1))*(delta*q - delta + 1)/(1 + exp(-beta*(-R + T))) + delta**4*(1 - q)*(1 - 1/(N - 1))*(delta*q - delta + 1)/(1 + exp(-beta*(P/2 - R + T/2))) - delta**4*(1 - q)*(1 - 1/(N - 1))*(delta*q - delta)/(1 + exp(-beta*(P - R))) - delta**2*q*(delta*q - delta)/((1 + exp(-beta*(-S/2 + T/2)))*(N - 1)) + delta**2*q*(delta*q - delta + 1)/((1 + exp(-beta*(-S + T)))*(N - 1)) - delta**2*(1 - q)*(delta*q - delta)/(2*(N - 1)) + delta**2*(1 - q)*(delta*q - delta + 1)/((1 + exp(-beta*(-S/2 + T/2)))*(N - 1))

In [557]:
(expr.collect(delta - 1) - (f + s)).expand()

0

In [551]:
f  / (1 - 1/ (N - 1))

-delta**4*q*(delta*q - delta)/(1 + exp(-beta*(P/2 - R + T/2))) + delta**4*q*(delta*q - delta + 1)/(1 + exp(-beta*(-R + T))) + delta**4*(1 - q)*(delta*q - delta + 1)/(1 + exp(-beta*(P/2 - R + T/2))) - delta**4*(1 - q)*(delta*q - delta)/(1 + exp(-beta*(P - R)))

In [561]:
# expr.subs({delta:0.999, b:10, c:1, beta:1, q:.8, N:100}).round(5)

In [562]:
# expr2.subs({delta:0.999, b:10, c:1, beta:1, q:.8, N:100}).round(5)

In [563]:
expr2 = (sum(sum(np.multiply(x, rho.T))))

In [564]:
expr2.collect(delta - 1)

-delta**4*q*(1 - 1/(N - 1))*(delta*q - delta)/(1 + exp(-beta*(-P/2 + R - T/2))) + delta**4*q*(1 - 1/(N - 1))*(delta*q - delta + 1)/(1 + exp(-beta*(R - T))) + delta**4*(1 - q)*(1 - 1/(N - 1))*(delta*q - delta + 1)/(1 + exp(-beta*(-P/2 + R - T/2))) - delta**4*(1 - q)*(1 - 1/(N - 1))*(delta*q - delta)/(1 + exp(-beta*(-P + R))) + delta**2*q*(delta*q - delta + 1)/((1 + exp(-beta*(S - T)))*(N - 1)) - delta**2*q*(delta*q - delta)/((1 + exp(-beta*(S/2 - T/2)))*(N - 1)) - delta**2*(1 - q)*(delta*q - delta)/(2*(N - 1)) + delta**2*(1 - q)*(delta*q - delta + 1)/((1 + exp(-beta*(S/2 - T/2)))*(N - 1))

In [601]:
f2 = (1 - 1/ (N - 1)) * (num1 / (sym.exp(-beta  * (-P/2 + R - T/2)) + 1) + num2 / (sym.exp(-beta  * (R - T)) + 1)
                       + num3 / (sym.exp(-beta  * (-P/2 + R - T/2)) + 1) + num4 / (sym.exp(-beta  * (-P + R)) + 1))


s2 = (1 / (N - 1)) * (+ num21 / (sym.exp(-beta  * (S - T)) + 1) - num22 /  (sym.exp(-beta * (S/2 - T/2)) + 1)
                    - num23 / 2 + num24 / (sym.exp(-beta  * (S/2 - T/2)) + 1))

In [602]:
f2

(1 - 1/(N - 1))*(-delta**4*q*(delta*q - delta)/(1 + exp(-beta*(-P/2 + R - T/2))) + delta**4*q*(delta*q - delta + 1)/(1 + exp(-beta*(R - T))) + delta**4*(1 - q)*(delta*q - delta + 1)/(1 + exp(-beta*(-P/2 + R - T/2))) - delta**4*(1 - q)*(delta*q - delta)/(1 + exp(-beta*(-P + R))))

In [603]:
s2

(delta**2*q*(delta*q - delta + 1)/(1 + exp(-beta*(S - T))) - delta**2*q*(delta*q - delta)/(1 + exp(-beta*(S/2 - T/2))) - delta**2*(1 - q)*(delta*q - delta)/2 + delta**2*(1 - q)*(delta*q - delta + 1)/(1 + exp(-beta*(S/2 - T/2))))/(N - 1)

In [604]:
(expr2.collect(delta - 1) - (f2 + s2)).expand()

0

In [610]:
((f  / (1 - 1/ (N - 1))) / (f2 / (1 - 1/ (N - 1))))

(-delta**4*q*(delta*q - delta)/(1 + exp(-beta*(P/2 - R + T/2))) + delta**4*q*(delta*q - delta + 1)/(1 + exp(-beta*(-R + T))) + delta**4*(1 - q)*(delta*q - delta + 1)/(1 + exp(-beta*(P/2 - R + T/2))) - delta**4*(1 - q)*(delta*q - delta)/(1 + exp(-beta*(P - R))))/(-delta**4*q*(delta*q - delta)/(1 + exp(-beta*(-P/2 + R - T/2))) + delta**4*q*(delta*q - delta + 1)/(1 + exp(-beta*(R - T))) + delta**4*(1 - q)*(delta*q - delta + 1)/(1 + exp(-beta*(-P/2 + R - T/2))) - delta**4*(1 - q)*(delta*q - delta)/(1 + exp(-beta*(-P + R))))

In [606]:
mcode(((f  / (1 - 1/ (N - 1))) / (f2 / (1 - 1/ (N - 1)))).subs({delta:1}))

'(q^2/(1 + Exp[-beta*(-R + T)]) + q*(1 - q)/(1 + Exp[-beta*((1/2)*P - R + (1/2)*T)]) - q*(q - 1)/(1 + Exp[-beta*((1/2)*P - R + (1/2)*T)]) - (1 - q)*(q - 1)/(1 + Exp[-beta*(P - R)]))/(q^2/(1 + Exp[-beta*(R - T)]) + q*(1 - q)/(1 + Exp[-beta*(-1/2*P + R - 1/2*T)]) - q*(q - 1)/(1 + Exp[-beta*(-1/2*P + R - 1/2*T)]) - (1 - q)*(q - 1)/(1 + Exp[-beta*(-P + R)]))'

In [607]:
1 / np.sqrt(2)

0.7071067811865475

In [489]:
(q ** 2 - 2 * q * (q - 1)).expand()

-q**2 + 2*q

In [490]:
((q - 1) ** 2 - 2 * q * (q - 1)).expand()

1 - q**2

In [422]:
sym.solve(q ** 2 + q - 1)[1]

-sqrt(5)/2 - 1/2

In [423]:
-(np.sqrt(5) / 2) - 1/2

-1.618033988749895

In [435]:
sym.solve(q - ((1 - q) ** 2).expand())[1]

sqrt(5)/2 + 3/2

In [433]:
(np.sqrt(5) / 2) 

1.118033988749895

In [416]:
(f + s) / (f2 + s2)

((1 - 1/(N - 1))*(-delta**4*q*(delta*q - delta)/(1 + exp(-beta*(-b/2 + c))) + delta**4*q*(delta*q - delta + 1)/(1 + exp(-beta*c)) + delta**4*(1 - q)*(delta*q - delta + 1)/(1 + exp(-beta*(-b/2 + c))) - delta**4*(1 - q)*(delta*q - delta)/(1 + exp(-beta*(-b + c)))) + (delta**2*q*(delta*q - delta + 1)/(1 + exp(-beta*(b + c))) - delta**2*q*(delta*q - delta)/(1 + exp(-beta*(b/2 + c/2))) - delta**2*(1 - q)*(delta*q - delta)/2 + delta**2*(1 - q)*(delta*q - delta + 1)/(1 + exp(-beta*(b/2 + c/2))))/(N - 1))/((1 - 1/(N - 1))*(delta**4*q*(delta*q - delta + 1)/(exp(beta*c) + 1) - delta**4*q*(delta*q - delta)/(1 + exp(-beta*(b/2 - c))) - delta**4*(1 - q)*(delta*q - delta)/(1 + exp(-beta*(b - c))) + delta**4*(1 - q)*(delta*q - delta + 1)/(1 + exp(-beta*(b/2 - c)))) + (-delta**2*q*(delta*q - delta)/(1 + exp(-beta*(-b/2 - c/2))) + delta**2*q*(delta*q - delta + 1)/(1 + exp(-beta*(-b - c))) - delta**2*(1 - q)*(delta*q - delta)/2 + delta**2*(1 - q)*(delta*q - delta + 1)/(1 + exp(-beta*(-b/2 - c/2))))/(N -

In [409]:
(expr2.collect(delta - 1) - (f2 + s2)).expand()

0

In [439]:
eq = - delta ** 4 * q * (delta * q - delta) / (- delta ** 4 * q * (delta * q - delta) 
                                          - delta ** 4 * (1 - q) * (delta * q - delta)
                                         + delta ** 4 * (1 - q) * (delta * q - delta + 1)) - 1

In [443]:
eq.subs({delta:1}).factor()

-1/(q + 1)

In [225]:
from sympy import mathematica_code as mcode, symbols, sin

In [226]:
mcode(sum(sum(np.multiply(x, rho))).removeO())

'delta^4*q*(1 - delta)*(1 - 1/(N - 1))*(delta*q - delta)/((1 + Exp[-beta*(-1/2*b + c)])*(delta - 1)) - delta^4*q*(1 - delta)*(1 - 1/(N - 1))*(delta*q - delta + 1)/((1 + Exp[-beta*c])*(delta - 1)) - delta^4*(1 - delta)*(1 - q)*(1 - 1/(N - 1))*(delta*q - delta + 1)/((1 + Exp[-beta*(-1/2*b + c)])*(delta - 1)) + delta^4*(1 - delta)*(1 - q)*(1 - 1/(N - 1))*(delta*q - delta)/((1 + Exp[-beta*(-b + c)])*(delta - 1)) - delta^2*q*(1 - delta)*(delta*q - delta + 1)/((1 + Exp[-beta*(b + c)])*(N - 1)*(delta - 1)) + delta^2*q*(1 - delta)*(delta*q - delta)/((1 + Exp[-beta*((1/2)*b + (1/2)*c)])*(N - 1)*(delta - 1)) + (1/2)*delta^2*(1 - delta)*(1 - q)*(delta*q - delta)/((N - 1)*(delta - 1)) - delta^2*(1 - delta)*(1 - q)*(delta*q - delta + 1)/((1 + Exp[-beta*((1/2)*b + (1/2)*c)])*(N - 1)*(delta - 1))'

In [227]:
sum(sum(np.multiply(x, rho.T)))

-delta**4*q*(1 - delta)*(1 - 1/(N - 1))*(delta*q - delta + 1)/((delta - 1)*(exp(beta*c) + 1)) + delta**4*q*(1 - delta)*(1 - 1/(N - 1))*(delta*q - delta)/((1 + exp(-beta*(b/2 - c)))*(delta - 1)) + delta**4*(1 - delta)*(1 - q)*(1 - 1/(N - 1))*(delta*q - delta)/((1 + exp(-beta*(b - c)))*(delta - 1)) - delta**4*(1 - delta)*(1 - q)*(1 - 1/(N - 1))*(delta*q - delta + 1)/((1 + exp(-beta*(b/2 - c)))*(delta - 1)) + delta**2*q*(1 - delta)*(delta*q - delta)/((1 + exp(-beta*(-b/2 - c/2)))*(N - 1)*(delta - 1)) - delta**2*q*(1 - delta)*(delta*q - delta + 1)/((1 + exp(-beta*(-b - c)))*(N - 1)*(delta - 1)) + delta**2*(1 - delta)*(1 - q)*(delta*q - delta)/(2*(N - 1)*(delta - 1)) - delta**2*(1 - delta)*(1 - q)*(delta*q - delta + 1)/((1 + exp(-beta*(-b/2 - c/2)))*(N - 1)*(delta - 1))

In [448]:
rho

Matrix([
[                           1/2,           1/(exp(b*beta/2) + 1),         1/(1 + exp(-beta*c/2)), 1/(1 + exp(-beta*(-b/2 + c/2))),           1/(exp(b*beta/2) + 1),             1/(exp(b*beta) + 1), 1/(1 + exp(-beta*(-b/2 + c/2))),   1/(1 + exp(-beta*(-b + c/2))),         1/(1 + exp(-beta*c/2)), 1/(1 + exp(-beta*(-b/2 + c/2))),           1/(1 + exp(-beta*c)),   1/(1 + exp(-beta*(-b/2 + c))), 1/(1 + exp(-beta*(-b/2 + c/2))),   1/(1 + exp(-beta*(-b + c/2))),   1/(1 + exp(-beta*(-b/2 + c))),     1/(1 + exp(-beta*(-b + c)))],
[        1/(1 + exp(-b*beta/2)),                             1/2, 1/(1 + exp(-beta*(b/2 + c/2))),          1/(1 + exp(-beta*c/2)),                             1/2,           1/(exp(b*beta/2) + 1),          1/(1 + exp(-beta*c/2)), 1/(1 + exp(-beta*(-b/2 + c/2))), 1/(1 + exp(-beta*(b/2 + c/2))),          1/(1 + exp(-beta*c/2)),   1/(1 + exp(-beta*(b/2 + c))),            1/(1 + exp(-beta*c)),          1/(1 + exp(-beta*c/2)), 1/(1 + exp(-beta*(-b/2 + c/2))),       

In [209]:
((N - 2) / (N - 1)) * x[0, :]

Matrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -delta**4*q*(1 - delta)*(1 - 1/(N - 1))*(N - 2)*(delta*q - delta + 1)/((N - 1)*(delta - 1)), -delta**4*(1 - delta)*(1 - q)*(1 - 1/(N - 1))*(N - 2)*(delta*q - delta + 1)/((N - 1)*(delta - 1)), 0, 0, delta**4*q*(1 - delta)*(1 - 1/(N - 1))*(N - 2)*(delta*q - delta)/((N - 1)*(delta - 1)), delta**4*(1 - delta)*(1 - q)*(1 - 1/(N - 1))*(N - 2)*(delta*q - delta)/((N - 1)*(delta - 1))]])

In [210]:
x

Matrix([
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -delta**4*q*(1 - delta)*(1 - 1/(N - 1))*(delta*q - delta + 1)/(delta - 1), -delta**4*(1 - delta)*(1 - q)*(1 - 1/(N - 1))*(delta*q - delta + 1)/(delta - 1), 0, 0, delta**4*q*(1 - delta)*(1 - 1/(N - 1))*(delta*q - delta)/(delta - 1), delta**4*(1 - delta)*(1 - q)*(1 - 1/(N - 1))*(delta*q - delta)/(delta - 1)],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0,                                                                         0,                                                                               0, 0, 0,                                                                    0,                                                                          0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0,                                                                         0,                                                                               0, 0, 0,                                                                    0,                                                 

In [211]:
vMR

Matrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -delta**2*q*(1 - delta)*(delta*q - delta + 1)/(delta - 1), -delta**2*(1 - delta)*(1 - q)*(delta*q - delta + 1)/(delta - 1), 0, 0, delta**2*q*(1 - delta)*(delta*q - delta)/(delta - 1), delta**2*(1 - delta)*(1 - q)*(delta*q - delta)/(delta - 1)]])

In [ ]:
laplus(k) = sum(sum(x.*Rho));
laminus(k) = sum(sum(x.*Rho'));

In [195]:
x.shape

(16, 16)

In [75]:
i1 = 7

In [71]:
x[7, :]

Matrix([[0, 0, 0, 0, 0, 0, 0, -delta**2*(1 - delta)*(1 - q)*(delta*q - delta + 1)/((N - 1)*(delta - 1)), -delta**2*(1 - delta)*(1 - q)*(delta*q - delta + 1)/((N - 1)*(delta - 1)), -delta**2*(1 - delta)*(1 - q)*(delta*q - delta + 1)/((N - 1)*(delta - 1)), -delta**2*(1 - delta)*(1 - q)*(delta*q - delta + 1)/((N - 1)*(delta - 1)), 0, 0, 0, 0, 0]])

In [74]:
vRR[7], vRM[7], vMR[7]

(0, -delta**2*(1 - delta)*(1 - q)*(delta*q - delta + 1)/(delta - 1), 0)

In [71]:
alld, gtft, deltaV = np.random.random(3), np.random.random(3), 0.99

In [72]:
import evol_dynamics

In [85]:
evol_dynamics.stationary_for_16_states(alld, gtft, deltaV).round(3)

array([0.012, 0.008, 0.016, 0.012, 0.007, 0.005, 0.069, 0.048, 0.018,
       0.071, 0.025, 0.102, 0.011, 0.044, 0.105, 0.427])

In [86]:
v.reshape(1, 16).subs({delta:0.99})

Matrix([[0.011536959597899, 0.00811079417519925, 0.0164739265081158, 0.0115816152454082, 0.00704936469052941, 0.00495589375915072, 0.0687433079952453, 0.0483284020664595, 0.0175933158503918, 0.07133122062407, 0.0251219560833169, 0.10185571651903, 0.0108092546170965, 0.0438255831038631, 0.105408636375468, 0.427374052789112]])

In [30]:
s2

array([0.91236689, 0.64018716, 0.06849263])

In [445]:
A = np.array([[1, 2], [3, 4]])

In [447]:
A.T

array([[1, 3],
       [2, 4]])